# ICEBERG MIGRATOR Pickler

Once Iceberg Migrator updates are completed, the source code and files can be pickled using this notebook. This notebook is meant to be run from this location in the repo to package up the Iceberg Migrator files into lists that are picked and then written to a file. 

These files will be used by the Installer to install Iceberg Migrator.


In [15]:
import base64
import codecs
import os
import pickle

## STEP 1: Function definition

In [16]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

def encode(obj):
    pickled = pickle.dumps(obj)
    return codecs.encode(pickled, 'base64').decode()   

def get_files(path):
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f != ".DS_Store"]
    files.sort()
    for file in files:
        yield file

## STEP 2: Set directory paths

In [17]:
#get this repo's root dir
repo_root = os.path.dirname(os.getcwd())

#set pickle directory
pickle_dir = f"{repo_root}/pickles"

#set scripts/imgs directories
setup_dir = f"{repo_root}/iceberg-migrator/scripts"
streamlit_main = f"{repo_root}/iceberg-migrator/streamlit/main.py"
streamlit_env = f"{repo_root}/iceberg-migrator/streamlit/environment.yml"
streamlit_utils_dir = f"{repo_root}/iceberg-migrator/streamlit/utils"
streamlit_img_dir = f"{repo_root}/iceberg-migrator/streamlit/img"

## STEP 3: Add content of directory files to list

In [18]:
#setup
setup_setup_dict = {}
for file in get_files(setup_dir):
    with open(setup_dir + "/" + file, "rb") as current_file:
        setup_setup_dict[file] = current_file.read()
        
#streamlit_main
streamlit_main_dict = {}
with open(streamlit_main, "rb") as current_file:
    streamlit_main_dict["main.py"] = current_file.read()
    
#streamlit_env
streamlit_env_dict = {}
with open(streamlit_env, "rb") as current_file:
    streamlit_env_dict["environment.yml"] = current_file.read()
        
#streamlit_utils
streamlit_utils_dict = {}
for file in get_files(streamlit_utils_dir):
    with open(streamlit_utils_dir + "/" + file, "rb") as current_file:
        streamlit_utils_dict[file] = current_file.read()

#streamlit_img
streamlit_img_dict = {}
for file in get_files(streamlit_img_dir):
    with open(streamlit_img_dir + "/" + file, "rb") as current_file:
        encoded_img = base64.b64encode(current_file.read())
        streamlit_img_dict[file] = encoded_img

## STEP 4: Create pickle files

In [19]:
#write im setup pickle file
obj = {
    "setup" : setup_setup_dict,
    "streamlit_main" : streamlit_main_dict,
    "streamlit_env" : streamlit_env_dict,
    "streamlit_utils" : streamlit_utils_dict,
    "streamlit_imgs" : streamlit_img_dict,
}
encoded_objj = encode(obj)
with open(f"{pickle_dir}/setup.pickle", "w") as p:
    p.write(encoded_objj)     

## STEP 5: Decode pickles to verify contents (optional)
**Recommendation:** Copy each decoded string to a text editor for better readability.

In [ ]:
print(decode(encoded_objj))